# Day 17

In [13]:
from itertools import cycle

## Part 1

In [23]:
class RockPile:
 
    rock_shapes = {
        "line": [
            "0011110",
        ],
        "cross": [
            "0001000",
            "0011100",
            "0001000",
        ],
        "elbow": [
            "0000100",
            "0000100",
            "0011100",
        ],
        "column": [
            "0010000",
            "0010000",
            "0010000",
            "0010000",
        ],
        "square": [
            "0011000",
            "0011000",
        ]
    }
 
    def __init__(self, wind_directions: str):
        self.wind_direction = cycle(wind_directions)
        self.rock_shape = cycle(self.rock_shapes)
        self.pile: list[int] = []
 
    @property
    def height(self) -> int:
        return len(self.pile)
 
    def can_shift(self, direction: str, pile_level: int, rock: list[int]) -> bool:
        for r in range(len(rock)):
            if (
                    direction == ">" and
                    ((rock[r] & 1) or (rock[r] >> 1 & self.pile[pile_level + r]))
            ) or (
                    direction == "<" and
                    ((rock[r] & 2 ** 6) or (rock[r] << 1 & self.pile[pile_level + r]))
            ):
                return False
        return True
 
    def can_fall(self, pile_level: int, rock: list[int]) -> bool:
        for r in range(len(rock)):
            if ((pile_level + r >= len(self.pile) - 1) or
                    (rock[r] & self.pile[pile_level + r + 1])):
                return False
        return True
 
    def drop_next(self) -> None:
        rock = [int(i, 2) for i in self.rock_shapes[next(self.rock_shape)]]
        self.pile = [0] * (3 + len(rock)) + self.pile
        for pile_level in range(len(self.pile)):
            direction = next(self.wind_direction)
 
            if self.can_shift(direction, pile_level, rock):
                for r in range(len(rock)):
                    rock[r] = rock[r] >> 1 if direction == ">" else rock[r] << 1
 
            if not self.can_fall(pile_level, rock):
                for r in range(len(rock)):
                    self.pile[pile_level + r] = self.pile[pile_level + r] | rock[r]
                while self.pile[0] == 0:
                    self.pile.pop(0)
                return
 
    def dump(self) -> str:
        output = ""
        for r in self.pile:
            output += "|" + \
                      "".join(["#" if c == "1" else "." for c in format(r, "07b")]) + \
                      "|\n"
        output += "+-------+"
        return output

In [29]:
def part_1(filename) -> int:
    with open(filename) as input_text:
        for line in input_text:
            winds = line.strip('\n')

    pile = RockPile(winds)
    for _ in range(2022):
        pile.drop_next()
    return pile.height

In [30]:
assert (part_1('test_day17.txt') == 3068)

In [31]:
print(part_1('input_day17.txt'))

3235


## Part 2

In [32]:
def find_pattern(data: list[int]) -> tuple[list[int], list[int]]:
    for p in range(len(data)):
        sd = data[p:]
        for r in range(2, len(sd) // 2):
            if sd[0:r] == sd[r:2 * r]:
                if all([(sd[0:r] == sd[y:y + r]) for y in range(r, len(sd) - r, r)]):
                    return data[:p], data[p:p + r]
    return [], []

In [33]:
def part_2(filename) -> int:
    num_rocks = 1000000000000
    sample_size = 10000
    
    with open(filename) as input_text:
        for line in input_text:
            winds = line.strip('\n')
 
    pile = RockPile(winds)
 
    height_deltas = []
    for _ in range(sample_size):
        prev_height = pile.height
        pile.drop_next()
        height_deltas.append(pile.height - prev_height)
 
    preamble, repetition = find_pattern(height_deltas)
    p_len = len(preamble)
    r_len = len(repetition)
 
    return sum(preamble) \
        + sum(repetition) * ((num_rocks - p_len) // r_len) \
        + sum(repetition[:((num_rocks - p_len) % r_len)])

In [35]:
assert (part_2('test_day17.txt')==1514285714288)

In [36]:
print(part_2('input_day17.txt'))

1591860465110
